아트허브 끄롤링

In [ ]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
import pickle
from tqdm import tqdm
import sqlite3
from sqlalchemy import create_engine

In [137]:
!pip install tqdm

In [138]:
from tqdm import tqdm

In [2]:
# #아트허브 크롤링 테스트 ! 일단 255개만
# yearList = ['2018', '2012'] #test1
# monthList = ['01']

# #아트허브 전체 크롤링 약 29000개
# yearList = ['2018','2017','2016','2015','2014','2013','2012','2011','2011','2010','2009']
# monthList = ['01', '02','03','04','05','06','07','08','09','10','11','12']

In [3]:
# urlP = 'http://www.arthub.co.kr/sub01/board03_list.htm?'
# headers = {'User-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.106 Safari/537.36'}

밑에 코드는 아트허브 전시 링크 끄롤링

In [4]:
# urlList = []
# seed = 'http://www.arthub.co.kr/sub01/'

# for i in range(len(yearList)):
#     for j in range(len(monthList)):
#         url = urlP + 'k_date=' + yearList[i] + '&' + 'k_month=' + monthList[j]\
#                    + '&page=1'
#         print(url)
#         resp = requests.get(url, headers=headers)
#         html = BeautifulSoup(resp.content, 'lxml') # year+month 페이지 읽어오기

#         links = html.select('a.sub01_list_title') 
              
#         #year+month의 1페이지에 있는 전시 링크
#         for link in links:
#             urlList.append( (yearList[i]+monthList[j], link.get_text(), seed + link.get('href')) ) 
        
#         pages = html.select('a.page') 
        
#         for page in pages: # 1페이지 외 다른 페이지 읽어오기
#             urlD = 'http://www.arthub.co.kr' + page.get('href')
#             print(urlD)        
#             respD = requests.get(urlD, headers=headers)
#             htmlD = BeautifulSoup(respD.content, 'lxml') 
            
#             links = html.select('a.sub01_list_title')                 
            
#             for link in links:
#                 urlList.append( (yearList[i]+monthList[j], link.get_text(), seed + link.get('href')) ) 

아트허브 링크를 피클로 저장

In [5]:
# # with open ('arthub_exhibit.p', 'wb') as fp:
# #     pickle.dump(urlList, fp)

아트허브 링크 파일 불러오기

In [6]:
# with open('arthub_exhibit.p', 'rb') as fp:
#     arthub_exhibit = pickle.load(fp)

In [7]:
# type(arthub_exhibit)

In [8]:
# len(arthub_exhibit)

get_info, get_text, get_img 함수들

In [9]:
def get_info(url):
    resp = requests.get(url,headers=headers)
    soup = BeautifulSoup(resp.content,'lxml')
    t3 = soup.select('table')[3]

    info = []
    for el in [el.next.next.next.next.next for el in t3.find_all('td',{'width':'84','height':'18','align':'left'})]: #전시정보 뽑아오기, tag가 드러움
        if el.string==None: #class 'bs4.element.Tag'
            info.append(el.text)
        else:                          #class 'bs4.element.NavigableString'
            info.append(el.string) 

    artists = list(map(lambda s:s.strip(), info[0].split(',')))
    articles = [td.next['href'] for td in t3.find_all('td',{'width':'510'})[1:]]

    del info[0]
    info.insert(0,artists)
    info.append(articles)
    
    return info


In [11]:
def get_text(url):
    resp = requests.get(url,headers=headers)
    soup = BeautifulSoup(resp.content,'lxml')
    
    ps = [p.text for p in soup.find_all('p',{'align':'justify'})]
    text = ''.join(ps)
    
    return text.strip()

In [12]:
def get_img(url):
    resp = requests.get(url,headers=headers)
    soup = BeautifulSoup(resp.content,'lxml')
    
    ps = soup.select('td[colspan=3] > div > p')
    ps2 = re.findall(r'src=[\S]*', str(ps))
    
    ps2 = re.sub(r'\[',"",str(ps2))
    ps2 = re.sub(r'\'src="',"",str(ps2))
    ps2 = re.sub(r'"\'',"",str(ps2))
    ps2 = re.sub(r'\]',"",str(ps2))
    if str(ps2) and not "http://arthub.co.kr" in str(ps2):
        ps2 = "http://arthub.co.kr" + str(ps2)
    
    return str(ps2)

index x부터 y까지의 아트허브 전시 내용을 크롤링해서 arts 리스트에 append.. 
근데 메모리가 너무 많이 먹어서 18000개에서 오류..
바로바로 sql로 저장하는 함수로 바꿀 필요가 있어보인다..

In [ ]:
# arts = []

# def get_arthub(x,y):
#     try:
#         for exhibit in arthub_exhibit[x:y]:
        
#             info = get_info(exhibit[2])
#             text = get_text(exhibit[2])
#             img = get_img(exhibit[2])
        
#             temp = [*exhibit,*info,text,img]
    
#             arts.append(temp)
            
#             print(len(arts))    
            
#     except Exception as e:
#         print("error point : " + str(len(arts)) + "\n" + e)
        
#         return get_arthub(arts, len(arts))
        
#     return arts

# get_arthub(arts, 0, 4999)

In [14]:
# len(arts)

arts 리스트를 판다스 데이터 프레임으로 변환

In [15]:
# data = pd.DataFrame(arts,columns=['연월','전시제목','링크','전시작가','전시일정','초대일시','관람시간','전시장소','주소','연락처','홈페이지','아티클','텍스트','이미지'])    

In [ ]:
# data["이미지"]

In [279]:
# data

,연월,전시제목,링크,전시작가,전시일정,초대일시,관람시간,전시장소,주소,연락처,홈페이지,아티클,텍스트,이미지
0,201801,천 년의 꽃 Millennium Flowers - 최수정展 :: Photography,http://www.arthub.co.kr/sub01/board03_view.htm...,[최수정(Choi Soojoung 崔秀禎)],2018-01-02 ▶ 2018-01-29,없음,Open 0:00 ~ Close 24:00,아트허브 온라인 갤러리(ARTHUB Online Gallery) 다른전시 보기,온라인 스페이스(Online Space),arthub2@naver.com,www.arthub.co.kr,[],● 천 년의 꽃 Millennium Flowers\n 전시되는 “천 년...,http://arthub.co.kr/admin/admin_board_01/data/...
1,201801,jig - 허정展 :: Printing,http://www.arthub.co.kr/sub01/board03_view.htm...,[허정(Huh Jung 許정)],2018-01-03 ▶ 2018-01-15,2018-01-03 PM 5:00,Open 10:00 ~ Close 19:00,인사아트스페이스(INSA ART SPACE) 다른전시 보기,서울 종로구 인사동길 56,02-734-1333,www.insaartspace.com,[http://nownews.seoul.co.kr/news/newsView.php?...,● 빈 곳(虛)에 정하기(定)\n Mathesis 와 Mimesis 사...,http://arthub.co.kr/admin/admin_board_01/data/...
2,201801,이공일팟 :: Painting,http://www.arthub.co.kr/sub01/board03_view.htm...,"[구성욱, 김정옥, 박주호, 방지영, 신혜진, 유명희, 장숙희, 김빛나, 김우중, ...",2018-01-03 ▶ 2018-01-09,없음,Open 10:00 ~ Close 18:00,토포하우스(TOPOHAUS) 다른전시 보기,서울 종로구 인사동11길 6,02-734-7555,topohaus.modoo.at,[],● 공(供) 스튜디오\n 인(人)과 공(共)이 합하여 이루어진 공(供)...,http://arthub.co.kr/admin/admin_board_01/data/...
3,201801,보이지 않는 것을 공기 중에 배열하는 기술 :: Various,http://www.arthub.co.kr/sub01/board03_view.htm...,"[금민정, 김순임, 김진희, 문연욱, 서혜영, 이예승, 조재영, 조준용]",2018-01-04 ▶ 2018-02-13,없음,Open 10:00 ~ Close 18:30(월요일 휴관),일우 스페이스(ILWOO SPACE) 다른전시 보기,서울시 중구 서소문동 41-3 대한항공빌딩 1F,02-753-6502,www.ilwoo.org,"[http://news1.kr/articles/?3199361, http://www...",인간과 사물을 포함한 세계는 우리의 망막에 맺히는 ‘보이는 것’과 안감과도 같이 그...,http://arthub.co.kr/admin/admin_board_01/data/...
4,201801,Vanishing Point - 이강훈展 :: Conceptual Art,http://www.arthub.co.kr/sub01/board03_view.htm...,[이강훈(Lee Kanghoon 李康熏)],2018-01-05 ▶ 2018-01-17,없음,Open 11:00 ~ Close 18:00(월요일 휴관),예술공간 서:로 다른전시 보기,서울시 용산구 신흥로 36길 6,seoro-art@naver.com,blog.naver.com/seoro-art,[],● Vanishing Point\n 관계성은 인간의 본질이다. 자아는 ...,http://arthub.co.kr/admin/admin_board_01/data/...
5,201801,An die Musik - 서혜례展 :: Painting,http://www.arthub.co.kr/sub01/board03_view.htm...,[서혜례(Suhr Haerae)],2018-01-06 ▶ 2018-01-27,2018-01-13 PM 5:00,Open 10:00 ~ Close 21:00(일요일 휴관),충정각 다른전시 보기,서울시 서대문구 충정로 2길 8,02-313-0424,,[],● An die Musik \n 음악을 좋아하는 작가의 음악적 감수성과...,
6,201801,유년의 뜰전 - 장애란展 :: Painting,http://www.arthub.co.kr/sub01/board03_view.htm...,[장애란(Jang Aeran)],2018-01-07 ▶ 2018-01-30,2018-01-13 PM 4:00,Open 11:00 ~ Close 18:00(월요일 휴관),카라스갤러리(KARA’S GALLERY) 다른전시 보기,서울 용산구 회나무로13길 34,02-6349-0810,www.karasgallery.org,[],"● 유년의 뜰, 창조적 퇴행\n - 존재론적 닮기로서의 아이되기\n...",http://arthub.co.kr/admin/admin_board_01/data/...
7,201801,채한리展 :: Painting,http://www.arthub.co.kr/sub01/board03_view.htm...,[채한리(Chae Hanlee)],2018-01-09 ▶ 2018-02-05,없음,Open 0:00 ~ Close 24:00,아트허브 온라인 갤러리(ARTHUB Online Gallery) 다른전시 보기,온라인 스페이스(Online Space),arthub2@naver.com,www.arthub.co.kr,[],● 채한리展\n 가족이나 친지 혹은 나 자신의 사진을 들여다보며 그리움...,http://arthub.co.kr/admin/admin_board_01/data/...
8,201801,이모젠 커닝햄展 :: Photography,http://www.arthub.co.kr/sub01/board03_view.htm...,[이모젠 커닝햄(Imogen Cunningham)],2018-01-09 ▶ 2018-02-22,없음,"Open 10:00 ~ Close 18:00(토 11:00~, 일공휴일 휴관)",아트스페이스 J(ART SPACE J) 다른전시 보기,경기도 성남시 분당구 정자동 159-3,031-712-7528,www.artspacej.com,[http://www.kgnews.co.kr/news/articleView.html...,아트스페이스 J에서는 2018년을 여는 첫 전시로 미국을 대표하는 여류사진가 이모젠...,http://arthub.co.kr/admin/admin_board_01/data/...
9,201801,"수직충동, 수평충동 :: Various",http://www.arthub.co.kr/sub01/board03_view.htm...,"[강운, 권부문, 김용수, 김윤종, 김인배, 박석원, 박찬민, 박현기, 심문필, 원...",2018-01-09 ▶ 2018-04-29,2018-01-22 PM 5:00,Open 10:00 ~ Close 18:00(월요일 휴관),대구미술관(DAEGU ART MUSEUM) 다른전시 보기,대구시 수성구 미술관로 40,053-790-3000,www.daeguartmuseum.org,[http://www.yeongnam.com/mnews/newsview.do?mod...,"<2018 소장품전_수직충동, 수평충동>은 시각예술의 기초적인 조형요소인 ‘수직’과...",http://arthub.co.kr/admin/admin_board_01/data/...


sqlite로 판다스 데이터 프레임을 저장하는 거 시도 >> 실패함..

In [301]:
# con = sqlite3.connect("D:/taeking/bigdata/arthub.db")?
# data.to_sql("arthub255", con, index=False, if_exists='replace')

연산량 많으니까 다음번을 위해 5000개의 arthub 데이터를 csv로 저장하고 불러오기

In [16]:
# arthub_csv = data.to_csv('D:/taeking/bigdata/arthub5000.csv', encoding='utf-8', index=False)

In [17]:
# arthub_csv = pd.read_csv('D:/taeking/bigdata/arthub5000.csv', encoding='utf-8')

In [18]:
# arthub_csv

In [19]:
# len(arthub_csv)

In [25]:
# arthub_csv.sample(5)

백 박사님 아트허브 데이터 test1, test2

In [2]:
test1 = pd.read_excel('D:/taeking/bigdata/test.xlsx', encoding='utf-8')

In [3]:
test2 = pd.read_excel('D:/taeking/bigdata/test2.xlsx', encoding='utf-8')

In [4]:
len(test1)

8035

In [5]:
len(test2)

18309

In [213]:
test2.head(1)

,연월,전시제목,링크,전시작가,전시일정,초대일시,관람시간,전시장소,주소,연락처,홈페이지,아티클,텍스트
0,200801,이다展 :: Painting,http://www.arthub.co.kr/sub01/board03_view.htm...,['이다(Ree Da)'],2006-06-07 ▶ 2006-09-17,NaN,Open 10:00 ~ Close 18:00,유아트스페이스(Yoo Art Space) 다른전시 보기,서울시 강남구 청담동 101-6,02-541-8321,www.yooartspace.com,[''],"● \n 욕망 재현의 중의적 알리바이 : 색(色)은 색이요, 욕(慾..."


gensim에서 doc2vec을 이용해보자.

In [6]:
!pip install gensim

In [7]:
import gensim
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.test.utils import get_tmpfile
from gensim.corpora import Dictionary

D:\Anaconda3\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [10]:
type(test2.텍스트)

pandas.core.series.Series

(형태소 분석을 위해 test2 텍스트를 하나의 코퍼스로 만들어봄) >> 이제 필요없는 과정..

In [209]:
# text = ""
# count = 0
# for txt in test2.텍스트:
#     text += str(txt)
#     count+=1
#     print(count)

In [210]:
# with open('D:/taeking/bigdata/test2text.p', 'wb') as fp:
#     pickle.dump(text, fp)

In [211]:
# with open('D:/taeking/bigdata/test2text.p', 'rb') as fp:
#     corpus = pickle.load(fp)

In [212]:
# del text

In [8]:
from konlpy.tag import Kkma
kkma = Kkma()

In [208]:
# kkma.nouns(test2['텍스트'][0])

여기서부터가 doc2vec !!!!

In [10]:
from gensim.models import doc2vec
from collections import namedtuple
from gensim.test.utils import get_tmpfile

doc2vec input 형태로 만들기 위해,
test2 텍스트 데이터에 tag를 붙여서 docs 리스트에 네임드튜플 형태로 넣어주자.

doc2vec 학습 input: doc[namedtuple(tag, words)] 느낌

In [11]:
# docs = []
# analyzeDoc = namedtuple('AnalyzedDocumnet', 'words tags')

In [37]:
# for i in range(len(test2['텍스트'])):
#     temp = str(test2['텍스트'][i])
#     words = str(temp.lower().split())
#     tags = [i]
#     docs.append(analyzeDoc(words,tags))

In [38]:
# model = doc2vec.Doc2Vec(docs, vector_size = 100, window = 300, min_count = 1, workers = 4)

위에서는 konlpy를 사용하지 않아서 텍스트가 한 음절씩 잘렸다..

In [42]:
# fname = get_tmpfile("D:/taeking/bigdata/doc2vec_test_model")

In [43]:
# model.save(fname)

In [48]:
# model = Doc2Vec.load(fname)

twitter를 이용하여 doc2vec을 이용해보자!

In [13]:
from gensim.models import doc2vec
from collections import namedtuple
from gensim.test.utils import get_tmpfile

In [214]:
docs = []
analyzeDoc = namedtuple('AnalyzedDocument', 'words tags')

test2 데이터 중 중복 삭제

In [120]:
print(len(test2))
test2.drop_duplicates(['링크'], inplace=True)
print(len(test2))

13194
13194


In [134]:
test2.reset_index(drop=True, inplace=True)
print(len(test2))
test2.tail(5)

13194


,연월,전시제목,링크,전시작가,전시일정,초대일시,관람시간,전시장소,주소,연락처,홈페이지,아티클,텍스트
13189,201312,반야심경 - 고산 최은철展 :: Calligraphy & Engraving,http://www.arthub.co.kr/sub01/board03_view.htm...,['최은철(Choi Eunchul)'],2013-12-17 ▶ 2013-12-28,2013-12-17 PM 5:00,Open 10:00 ~ Close 19:00,갤러리 신시(Gallery Seensee) 다른전시 보기,서울시 서초구 논현로 5길 31-14,070-4619-2827,www.iseensee.com,[''],고산 최은철은 동양미학 \n 철학 박사로 성균관 대학교를 졸업하였다. ...
13190,201312,Sonoration - 김범중展 :: Drawing,http://www.arthub.co.kr/sub01/board03_view.htm...,['김범중(Kim Beomjoong)'],2013-12-03 ▶ 2013-12-10,없음,Open 10:30 ~ Close 18:30,관훈갤러리(KWANHOON gallery) 다른전시 보기,서울시 종로구 관훈동 195,02-733-6469,www.kwanhoongallery.com,[''],NaN
13191,201312,곧(꽃)의 同行 - 김성현展 :: Glass,http://www.arthub.co.kr/sub01/board03_view.htm...,['김성현(Kim Sunghyun)'],2013-12-11 ▶ 2013-12-17,없음,Open 10:00 ~ Close 19:00,갤러리 가이아(GALERIE GAIA) 다른전시 보기,서울시 종로구 관훈동 145,02-733-3373,www.galerie-gaia.net,[''],"꽃상여의 인물, \n 동물 조각들을 공간 또한 상황으로 구축하여 죽음 ..."
13192,201312,긴 이야기가 있는 그림 - 한태희展 :: Painting,http://www.arthub.co.kr/sub01/board03_view.htm...,['한태희(Han Taehie)'],2013-12-06 ▶ 2013-12-19,없음,Open 12:00 ~ Close 19:00(월요일 휴관),대안공간 눈(alternative space noon) 다른전시 보기,경기도 수원시 팔달구 북수동 232-3,031-244-4519,www.spacenoon.co.kr,[''],마음속에서 나오는 \n 태초의 소리는 도형이나 언어라는 형태가 만들어지...
13193,201312,곡신불사 - 김석영展 :: Painting,http://www.arthub.co.kr/sub01/board03_view.htm...,['김석영(Kim Seokyoung)'],2013-12-07 ▶ 2014-01-15,없음,Open 10:00 ~ Close 20:00,"금보성아트센터 1, 2관 다른전시 보기",서울시 종로구 평창동 111-16,02-396-8744,NaN,[''],"무엇인가 요동치고 \n 술렁거리고, 무언가는 급한 속도로 내달리고, 또..."


In [143]:
from konlpy.tag import Twitter

In [144]:
twitter = Twitter()

In [146]:
for i in tqdm(range(len(test2['텍스트']))):
    temp = str(test2['텍스트'][i])
    words = twitter.morphs(temp)
    tags = [i]
    docs.append(analyzeDoc(words,tags))



  0%|                                                                                        | 0/13194 [00:00<?, ?it/s]

  0%|                                                                                | 3/13194 [00:00<15:29, 14.19it/s]

  0%|                                                                                | 4/13194 [00:00<17:55, 12.27it/s]

  0%|                                                                                | 7/13194 [00:00<13:35, 16.17it/s]

  0%|                                                                                | 9/13194 [00:00<14:20, 15.32it/s]

  0%|                                                                               | 14/13194 [00:00<11:32, 19.02it/s]

  0%|                                                                               | 17/13194 [00:00<10:48, 20.32it/s]

  0%|▏                                                                              | 22/13194 [00:00<09:25, 23.29it/s]

  0%|▏                        

  2%|█▏                                                                            | 199/13194 [00:10<11:21, 19.07it/s]

  2%|█▏                                                                            | 204/13194 [00:10<11:29, 18.83it/s]

  2%|█▏                                                                            | 207/13194 [00:10<11:29, 18.84it/s]

  2%|█▏                                                                            | 210/13194 [00:11<11:29, 18.84it/s]

  2%|█▎                                                                            | 213/13194 [00:11<11:28, 18.84it/s]

  2%|█▎                                                                            | 215/13194 [00:11<11:32, 18.75it/s]

  2%|█▎                                                                            | 217/13194 [00:11<11:35, 18.67it/s]

  2%|█▎                                                                            | 220/13194 [00:11<11:34, 18.67it/s]

  2%|█▎                         

  3%|██▎                                                                           | 400/13194 [00:21<11:15, 18.94it/s]

  3%|██▍                                                                           | 404/13194 [00:21<11:12, 19.02it/s]

  3%|██▍                                                                           | 408/13194 [00:21<11:11, 19.03it/s]

  3%|██▍                                                                           | 411/13194 [00:21<11:16, 18.90it/s]

  3%|██▍                                                                           | 414/13194 [00:21<11:18, 18.84it/s]

  3%|██▍                                                                           | 418/13194 [00:22<11:14, 18.93it/s]

  3%|██▍                                                                           | 421/13194 [00:22<11:13, 18.96it/s]

  3%|██▌                                                                           | 424/13194 [00:22<11:13, 18.97it/s]

  3%|██▌                        

  5%|███▋                                                                          | 623/13194 [00:31<10:44, 19.52it/s]

  5%|███▋                                                                          | 626/13194 [00:32<10:43, 19.53it/s]

  5%|███▋                                                                          | 630/13194 [00:32<10:41, 19.57it/s]

  5%|███▋                                                                          | 634/13194 [00:32<10:40, 19.62it/s]

  5%|███▊                                                                          | 639/13194 [00:32<10:37, 19.70it/s]

  5%|███▊                                                                          | 643/13194 [00:32<10:36, 19.71it/s]

  5%|███▊                                                                          | 646/13194 [00:32<10:35, 19.73it/s]

  5%|███▊                                                                          | 649/13194 [00:32<10:35, 19.75it/s]

  5%|███▊                       

  6%|████▉                                                                         | 833/13194 [00:42<10:24, 19.80it/s]

  6%|████▉                                                                         | 836/13194 [00:42<10:24, 19.80it/s]

  6%|████▉                                                                         | 840/13194 [00:42<10:23, 19.82it/s]

  6%|████▉                                                                         | 843/13194 [00:42<10:23, 19.82it/s]

  6%|█████                                                                         | 846/13194 [00:42<10:23, 19.80it/s]

  6%|█████                                                                         | 848/13194 [00:42<10:24, 19.76it/s]

  6%|█████                                                                         | 850/13194 [00:43<10:25, 19.72it/s]

  6%|█████                                                                         | 852/13194 [00:43<10:26, 19.69it/s]

  6%|█████                      

...

 91%|█████████████████████████████████████████████████████████████████████       | 11993/13194 [07:42<00:46, 25.93it/s]

 91%|█████████████████████████████████████████████████████████████████████       | 11998/13194 [07:42<00:46, 25.93it/s]

 91%|█████████████████████████████████████████████████████████████████████▏      | 12004/13194 [07:42<00:45, 25.93it/s]

 91%|█████████████████████████████████████████████████████████████████████▏      | 12009/13194 [07:42<00:45, 25.94it/s]

 91%|█████████████████████████████████████████████████████████████████████▏      | 12013/13194 [07:43<00:45, 25.93it/s]

 91%|█████████████████████████████████████████████████████████████████████▏      | 12018/13194 [07:43<00:45, 25.93it/s]

 91%|█████████████████████████████████████████████████████████████████████▏      | 12022/13194 [07:43<00:45, 25.94it/s]

 91%|█████████████████████████████████████████████████████████████████████▎      | 12026/13194 [07:43<00:45, 25.94it/s]

 91%|███████████████████████████

 93%|██████████████████████████████████████████████████████████████████████▋     | 12277/13194 [07:52<00:35, 26.00it/s]

 93%|██████████████████████████████████████████████████████████████████████▋     | 12281/13194 [07:52<00:35, 25.99it/s]

 93%|██████████████████████████████████████████████████████████████████████▊     | 12284/13194 [07:52<00:35, 25.99it/s]

 93%|██████████████████████████████████████████████████████████████████████▊     | 12289/13194 [07:52<00:34, 26.00it/s]

 93%|██████████████████████████████████████████████████████████████████████▊     | 12294/13194 [07:52<00:34, 26.00it/s]

 93%|██████████████████████████████████████████████████████████████████████▊     | 12298/13194 [07:52<00:34, 26.00it/s]

 93%|██████████████████████████████████████████████████████████████████████▊     | 12302/13194 [07:53<00:34, 26.00it/s]

 93%|██████████████████████████████████████████████████████████████████████▉     | 12307/13194 [07:53<00:34, 26.00it/s]

 93%|███████████████████████████

 95%|████████████████████████████████████████████████████████████████████████▍   | 12573/13194 [08:01<00:23, 26.10it/s]

 95%|████████████████████████████████████████████████████████████████████████▍   | 12579/13194 [08:01<00:23, 26.10it/s]

 95%|████████████████████████████████████████████████████████████████████████▍   | 12584/13194 [08:02<00:23, 26.10it/s]

 95%|████████████████████████████████████████████████████████████████████████▌   | 12590/13194 [08:02<00:23, 26.11it/s]

 95%|████████████████████████████████████████████████████████████████████████▌   | 12594/13194 [08:02<00:22, 26.11it/s]

 95%|████████████████████████████████████████████████████████████████████████▌   | 12598/13194 [08:02<00:22, 26.11it/s]

 96%|████████████████████████████████████████████████████████████████████████▌   | 12602/13194 [08:02<00:22, 26.11it/s]

 96%|████████████████████████████████████████████████████████████████████████▌   | 12606/13194 [08:02<00:22, 26.11it/s]

 96%|███████████████████████████

 97%|██████████████████████████████████████████████████████████████████████████  | 12853/13194 [08:11<00:13, 26.15it/s]

 97%|██████████████████████████████████████████████████████████████████████████  | 12857/13194 [08:11<00:12, 26.15it/s]

 97%|██████████████████████████████████████████████████████████████████████████  | 12861/13194 [08:11<00:12, 26.15it/s]

 98%|██████████████████████████████████████████████████████████████████████████  | 12867/13194 [08:11<00:12, 26.15it/s]

 98%|██████████████████████████████████████████████████████████████████████████▏ | 12871/13194 [08:12<00:12, 26.16it/s]

 98%|██████████████████████████████████████████████████████████████████████████▏ | 12875/13194 [08:12<00:12, 26.16it/s]

 98%|██████████████████████████████████████████████████████████████████████████▏ | 12879/13194 [08:12<00:12, 26.15it/s]

 98%|██████████████████████████████████████████████████████████████████████████▏ | 12884/13194 [08:12<00:11, 26.16it/s]

 98%|███████████████████████████

100%|███████████████████████████████████████████████████████████████████████████▊| 13161/13194 [08:21<00:01, 26.26it/s]

100%|███████████████████████████████████████████████████████████████████████████▊| 13166/13194 [08:21<00:01, 26.27it/s]

100%|███████████████████████████████████████████████████████████████████████████▊| 13172/13194 [08:21<00:00, 26.27it/s]

100%|███████████████████████████████████████████████████████████████████████████▉| 13176/13194 [08:21<00:00, 26.27it/s]

100%|███████████████████████████████████████████████████████████████████████████▉| 13182/13194 [08:21<00:00, 26.28it/s]

100%|███████████████████████████████████████████████████████████████████████████▉| 13187/13194 [08:21<00:00, 26.28it/s]

100%|████████████████████████████████████████████████████████████████████████████| 13194/13194 [08:21<00:00, 26.29it/s]



test2 twitter 분석한 거 저장하고 불러오기

In [11]:
!pip install dill

In [147]:
import dill

# with open('D:/taeking/bigdata/test2_Kkma_results_dropdup.dill', 'wb') as fp:
#     dill.dump(docs, fp)

with open('D:/taeking/bigdata/test2_Kkma_results_dropdup.dill', 'rb') as fp:
    docs = dill.load(fp)

In [64]:
# docs[0].words

In [113]:
# model_with_kkma = doc2vec.Doc2Vec(docs, vector_size = 100, window = 300, min_count = 1, workers = 4)

In [66]:
# fname = get_tmpfile("D:/taeking/bigdata/test2_doc2vec_kkma_model")
# model_with_kkma.save(fname)
# model_with_kkma = Doc2Vec.load(fname)

In [67]:
# model_with_kkma.delete_temporary_training_data(keep_doctags_vectors=True, keep_inference=True)

In [68]:
# tokens = kkma.nouns(test1['텍스트'][0])
# new_vector = model_with_kkma.infer_vector(tokens)
# sims = model_with_kkma.docvecs.most_similar([new_vector])
# sims

D:\Anaconda3\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[(8476, 0.9413787722587585),
 (8968, 0.9378604292869568),
 (7470, 0.9334474802017212),
 (14154, 0.9276226758956909),
 (10164, 0.926257848739624),
 (14520, 0.9242078065872192),
 (17127, 0.9227869510650635),
 (11247, 0.9220483899116516),
 (13249, 0.9190245866775513),
 (15401, 0.9172438383102417)]

In [69]:
# test1['링크'][0]

In [70]:
# test2['링크'][956]

In [76]:
# tokens = kkma.nouns(test2['텍스트'][84])
# new_vector = model_with_kkma.infer_vector(tokens)
# sims = model_with_kkma.docvecs.most_similar([new_vector])
# sims

D:\Anaconda3\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[(84, 0.9568134546279907),
 (17533, 0.9483017921447754),
 (14550, 0.9482304453849792),
 (516, 0.9444377422332764),
 (9270, 0.9351193904876709),
 (1136, 0.9321909546852112),
 (17311, 0.9236737489700317),
 (12359, 0.9226875901222229),
 (4607, 0.9226473569869995),
 (10503, 0.9194214344024658)]

위에 방법이 틀린거 같아서 아래 방법으로 학습을 시켜봄

epochs가 커질수록 성능이 좋아지나?

In [148]:
max_epochs = 20
vec_size = 200
alpha = 0.025

In [149]:
test2_kkma_model2 = doc2vec.Doc2Vec(vector_size=vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                dm =1)

In [150]:
test2_kkma_model2.build_vocab(docs)

In [151]:
for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    test2_kkma_model2.train(docs,
                total_examples=test2_kkma_model2.corpus_count,
                epochs=test2_kkma_model2.iter)
    # decrease the learning rate
    test2_kkma_model2.alpha -= 0.0002
    # fix the learning rate, no decay
    test2_kkma_model2.min_alpha = test2_kkma_model2.alpha

test2_kkma_model2.save("D:/taeking/bigdata/test2_kkma_model2_dropdup.model")
print("Model Saved")

iteration 0


D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  """


iteration 1
iteration 2
iteration 3
iteration 4
iteration 5
iteration 6
iteration 7
iteration 8
iteration 9
iteration 10
iteration 11
iteration 12
iteration 13
iteration 14
iteration 15
iteration 16
iteration 17
iteration 18
iteration 19
Model Saved


In [164]:
test2['텍스트'][2]

"이봉열의 평면 모노크롬 \n        회화는 68년 첫 개인전으로부터 일별하자면 거의 10년마다 한번씩 변모를 겪어 온 것으로 생각된다. 대개 70년대의 경향은 평면이라는 \n        단위 공간을 어떻게 분절하고 표면을 현전시킬 것인지에 역점을 두면서 특히 '격자구조'에 관심을 보였다면 80년대에는 탈(脫)격자 \n        내지는 격자 해체적 변주를 시도했던 것으로 볼 수 있다. 이어서 90년대의 10년간은 격자를 완전히 떠나 '몸'의 개념을 중심으로 \n        화면과 작가 자신을 일체화하는 데 관심을 쏟아 온 것으로 생각된다. 이번 개인전은 90년대 초 갤러리현대 초대전 이후 거반 10년사를 \n        마감하는 개인전이어서 그의 개인사(史)로 보았을 때 어느 때보다 시사하는 바가 크다고 할 수 있다. 근작들은 무엇보다 이전에 \n        이룩했던 성과를 회고하면서 이를 흔적으로 남기는 한편 과거의 잔흔을 거의 알아 볼 수 없으리만큼 지워진 단일 평면을 강조하면서 \n        이를 확인시키듯 명제 또한 <<지워진 공간>> 또는 <<무제 공간>>으로 이름부치고 \n        있어 주목된다. 이는 필시 다난했던 90년대의 바로 그 자신의 초상이자 내심을 응축해 놓은 것으로 볼 수 있을 것이다.\n        이미 앞에서 근작들에 관한 이해의 틀을 '화면과 작가의 몸의 일체화'라 적시했지만 여기서 그의 최근 10년사에서 <몸Body>의 \n        문제가 부상된 것은 대단히 중요한 의미를 갖는 것으로 생각된다. 그래서 이 점을 그의 공간 해석의 전 과정을 통해 <격자Grid>와 \n        대비시켜 자세히 기술해 둘 필요를 느낀다. 먼저 그가 화면을 다루는 데에는 화면이 갖고 있는 평면 개념을 시대적 상황과의 근접 \n        관계에서 다루려 했다는 것을 전제해 둘 필요가 있다. 공간의 문제만 해도 그렇다. 그는 일찌기 자신의 세계를 설정하는 데 있어서 \n        무엇보다 <공간>의 문제가 주요 

In [179]:
test_data = twitter.morphs(test2['텍스트'][2])

In [177]:
# test_data

In [178]:
new_vector = test2_kkma_model2.infer_vector(test_data)
sims = test2_kkma_model2.docvecs.most_similar([new_vector])
sims

D:\Anaconda3\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[(4, 0.8212549090385437),
 (3885, 0.5887981057167053),
 (5361, 0.5737881660461426),
 (5012, 0.5689372420310974),
 (1269, 0.5638279914855957),
 (1999, 0.5637192726135254),
 (4084, 0.5626730918884277),
 (4092, 0.5610911846160889),
 (9033, 0.5594266057014465),
 (9553, 0.5544693470001221)]

In [207]:
for i in range(50):
    test_data = twitter.morphs(test2['텍스트'][i])
    
    new_vector = test2_kkma_model2.infer_vector(test_data)
    sims = test2_kkma_model2.docvecs.most_similar([new_vector], topn=3)
    
    print(i, "■",  test2['전시제목'][i], test2['링크'][i], ">>", end="\n")
    for j in range(len(sims)):
        print(j+1, "위: ", test2['전시제목'][sims[j][0]], test2['링크'][sims[j][0]], "===", int(sims[j][1]*100), end="% \n")
    print(end="\n-----------------------------------------------------------------------------------\n")

D:\Anaconda3\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


0 ■ 이다展 :: Painting http://www.arthub.co.kr/sub01/board03_view.htm?No=394 >>
1 위:  이다展 :: Painting http://www.arthub.co.kr/sub01/board03_view.htm?No=394 === 88% 
2 위:  김선형展 :: Painting http://www.arthub.co.kr/sub01/board03_view.htm?No=10106 === 45% 
3 위:  興 - 이혜경展 :: Painting http://www.arthub.co.kr/sub01/board03_view.htm?No=6067 === 45% 

-----------------------------------------------------------------------------------
1 ■ landscape - 주명덕展 :: Photography http://www.arthub.co.kr/sub01/board03_view.htm?No=613 >>
1 위:  興 - 이혜경展 :: Painting http://www.arthub.co.kr/sub01/board03_view.htm?No=6067 === 67% 
2 위:  landscape - 주명덕展 :: Photography http://www.arthub.co.kr/sub01/board03_view.htm?No=613 === 67% 
3 위:  김선형展 :: Painting http://www.arthub.co.kr/sub01/board03_view.htm?No=10106 === 67% 

-----------------------------------------------------------------------------------
2 ■ 이봉열展 :: Painting http://www.arthub.co.kr/sub01/board03_view.htm?No=4680 >>
1 위:  이봉열展 :: Painting http://www.art

18 ■ 노정란展 :: Painting http://www.arthub.co.kr/sub01/board03_view.htm?No=328 >>
1 위:  노정란展 :: Painting http://www.arthub.co.kr/sub01/board03_view.htm?No=328 === 75% 
2 위:  김선형展 :: Painting http://www.arthub.co.kr/sub01/board03_view.htm?No=10106 === 61% 
3 위:  제8회 갤러리아쿠아 신진작가 구상展 :: Painting http://www.arthub.co.kr/sub01/board03_view.htm?No=9200 === 61% 

-----------------------------------------------------------------------------------
19 ■ 수묵의 결을 따라서 - 구지회展 :: Painting http://www.arthub.co.kr/sub01/board03_view.htm?No=4373 >>
1 위:  수묵의 결을 따라서 - 구지회展 :: Painting http://www.arthub.co.kr/sub01/board03_view.htm?No=4373 === 74% 
2 위:  여름향기 :: Painting http://www.arthub.co.kr/sub01/board03_view.htm?No=4664 === 62% 
3 위:  興 - 이혜경展 :: Painting http://www.arthub.co.kr/sub01/board03_view.htm?No=6067 === 59% 

-----------------------------------------------------------------------------------
20 ■ 방정아展 :: Painting http://www.arthub.co.kr/sub01/board03_view.htm?No=585 >>
1 위:  방정아展 :: Painting ht

36 ■ Practisculpture :: Sculpture & Installation http://www.arthub.co.kr/sub01/board03_view.htm?No=4245 >>
1 위:  Practisculpture :: Sculpture & Installation http://www.arthub.co.kr/sub01/board03_view.htm?No=4245 === 86% 
2 위:  My gratitude - 조희수展 :: Painting http://www.arthub.co.kr/sub01/board03_view.htm?No=11001 === 56% 
3 위:  김선형展 :: Painting http://www.arthub.co.kr/sub01/board03_view.htm?No=10106 === 56% 

-----------------------------------------------------------------------------------
37 ■ 정물산책 - 류제비展 :: Painting http://www.arthub.co.kr/sub01/board03_view.htm?No=2287 >>
1 위:  정물산책 - 류제비展 :: Painting http://www.arthub.co.kr/sub01/board03_view.htm?No=2287 === 79% 
2 위:  My gratitude - 조희수展 :: Painting http://www.arthub.co.kr/sub01/board03_view.htm?No=11001 === 56% 
3 위:  김선형展 :: Painting http://www.arthub.co.kr/sub01/board03_view.htm?No=10106 === 56% 

-----------------------------------------------------------------------------------
38 ■ 차규선展 :: Painting http://www.arthub.co.kr/

In [ ]:
# CBOW (syntactic task에 강점), Skip-gram (semantic task에 강점, 학습이 느림..)
# word embedding에서 hidden layer 값은 100~300
# 